In [2]:
from sqlalchemy import create_engine, text

engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)

In [3]:
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(text("INSERT INTO some_table (x, y) VALUES (:x, :y)"), [{"x": 1, "y": 2}, {"x": 3, "y": 4}])
    conn.commit()

2025-04-25 00:06:06,351 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-25 00:06:06,353 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2025-04-25 00:06:06,354 INFO sqlalchemy.engine.Engine [generated in 0.00253s] ()
2025-04-25 00:06:06,357 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2025-04-25 00:06:06,357 INFO sqlalchemy.engine.Engine [generated in 0.00100s] [(1, 2), (3, 4)]
2025-04-25 00:06:06,359 INFO sqlalchemy.engine.Engine COMMIT


# Result

`Result` 는 **순회가능(Iterable)**한 객체로 객체를 다른 형태로 **변환(transform)**시키는 작업이나 하나하나씩 순회하면서 확인하기 좋습니다.

In [4]:
with engine.begin() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    print(type(result))
    for row in result:
        print(f"x: {row.x}, y: {row.y}")

2025-04-25 00:08:21,655 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-25 00:08:21,656 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2025-04-25 00:08:21,657 INFO sqlalchemy.engine.Engine [generated in 0.00043s] ()
<class 'sqlalchemy.engine.cursor.CursorResult'>
x: 1, y: 2
x: 3, y: 4
2025-04-25 00:08:21,658 INFO sqlalchemy.engine.Engine COMMIT


## Row

**Result** 자료구조 안에 들어가있는 **Row** 도 독특한데요. 파이썬에서 간단하게 불변(immutable)적인 자료구조를 만들 수 있도록 도와주는 `namedtuple` 로 구성되어 있습니다. 따라서 아래와 같은 **언패킹**기능도 활용하여 조금 더 직관적으로 코드를 작성할 수 있습니다.

In [15]:
class Data():

    def __init__(self, x, y) -> None:
        self.x = x
        self.y = y
    
    def __repr__(self) -> str:
        return f"Data ({self.x}, {self.y})"

with engine.begin() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    """
    언패킹을 활용한 데이터에 쉽게 담기
    """
    data = [Data(*row) for row in result]
    print(data)

2025-04-25 00:21:09,976 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-25 00:21:09,978 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2025-04-25 00:21:09,979 INFO sqlalchemy.engine.Engine [cached since 768.3s ago] ()
[Data (1, 2), Data (3, 4)]
2025-04-25 00:21:09,980 INFO sqlalchemy.engine.Engine COMMIT


In [16]:
class Data():

    def __init__(self, **kwargs) -> None:
        self.x = kwargs["x"]
        self.y = kwargs["y"]
    
    def __repr__(self) -> str:
        return f"Data ({self.x}, {self.y})"

with engine.begin() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    """
    언패킹을 활용한 데이터에 쉽게 담기
    """
    data = [Data(**row._mapping) for row in result]
    print(data)

2025-04-25 00:21:13,280 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-25 00:21:13,282 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2025-04-25 00:21:13,283 INFO sqlalchemy.engine.Engine [cached since 771.6s ago] ()
[Data (1, 2), Data (3, 4)]
2025-04-25 00:21:13,284 INFO sqlalchemy.engine.Engine COMMIT
